In [131]:
import requests
import csv
from tqdm import tqdm
import urllib
from bs4 import BeautifulSoup as bs
import pandas as pd



base_url = 'http://www.jobkorea.co.kr/Search/?stext={}&tabType=recruit&Page_No={}'


def crawler(keyword, page_num):
    keyword = urllib.parse.quote(keyword)
    url = base_url.format(keyword,page_num)
    response = urllib.request.urlopen(url)
    soup = bs(response,'html.parser')

    #컬럼
    name = [element.text for element in soup.select("a.name")][:20]
    title = [element.text for element in soup.select("div.post-list-info > a.title")][:20]
    title = [element.replace("\n","").replace("\r","") for element in title]
    exp = [element.text for element in soup.select("p.option > span.exp")][:20]
    edu = [element.text for element in soup.select("p.option > span.edu")][:20]
    loc_short = [element.text for element in soup.select("p.option > span.short")][:20]
    loc_long = [element.text for element in soup.select("p.option > span.long")][:20]
    date = [element.text for element in soup.select("p.option > span.date")][:20]
    job = [element.text for element in soup.select("div.post-list-info > p.etc")][:20]
    df = pd.DataFrame({'기업명' : name, '내용' : title, '고용형태' : exp, '경력' : edu ,'주소1' : loc_short, '주소2' : loc_long, '기간' : date, '직무키워드' : job})
    return df

In [132]:
df = crawler("채용",1) #1페이지 내용 df로 지정
for i in range (2,100) :  # 2~100 페이지까지
 df_i = crawler("채용",i) 
 result = df.append(df_i)  # append
result = result.reset_index(drop = True) #인덱스 초기화

In [133]:
result.to_csv('잡코리아.csv', index=False, encoding="utf-8-sig")